In [19]:
import sqlite3
import os
import pandas as pd
pd.set_option('display.max_columns', None)

In [42]:
# load data from ../data/afl.db
conn = sqlite3.connect(os.path.join("..", "afl.db"))

In [48]:
player_stats = pd.read_sql_query("SELECT * FROM player_stats", conn)

In [37]:
game = pd.read_sql_query("SELECT * FROM game", conn)

In [36]:
player_stats.head()

,round,year,player_id,player_name,team,opponent,kicks,marks,hand_balls,disp,goals,behinds,hit_outs,tackles,rebounds,inside_50,clearances,clangers,frees_for,frees_against,brownlow,contested_possessions,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,percent_time_played,game_id
0,1,1996,720,Alastair Clarkson,Melbourne,Geelong,19,6,5,24,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172
1,1,1996,718,David Cockatoo-Collins,Melbourne,Geelong,3,0,2,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22172
2,1,1996,719,Don Cockatoo-Collins,Melbourne,Geelong,5,3,1,6,0,0,0,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172
3,1,1996,715,Greg Doyle,Melbourne,Geelong,8,4,1,9,4,1,7,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172
4,1,1996,712,Damien Gaspar,Melbourne,Geelong,4,3,3,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22172


In [38]:
game.head()

,round,agoals,game_id,unixtime,date,hteamid,ateam,tz,venue,localtime,winner,complete,hbehinds,ascore,roundname,year,is_final,hscore,abehinds,is_grand_final,updated,hgoals,timestr,hteam,winnerteamid,ateamid
0,1,20,1,1490257200,2017-03-23 19:20:00,3,Richmond,+11:00,M.C.G.,2017-03-23 19:20:00,Richmond,100,5,132,Round 1,2017,0,89,12,0,2017-04-15 15:59:16,14,Full Time,Carlton,14.0,14
1,1,15,2,1490345400,2017-03-24 19:50:00,4,Western Bulldogs,+11:00,M.C.G.,2017-03-24 19:50:00,Western Bulldogs,100,14,100,Round 1,2017,0,86,10,0,2017-04-15 15:59:16,12,Full Time,Collingwood,18.0,18
2,1,17,3,1490420100,2017-03-25 16:35:00,16,Port Adelaide,+11:00,S.C.G.,2017-03-25 16:35:00,Port Adelaide,100,10,110,Round 1,2017,0,82,8,0,2017-04-15 15:59:16,12,Full Time,Sydney,13.0,13
3,1,18,4,1490420100,2017-03-25 16:35:00,15,Melbourne,+11:00,Docklands,2017-03-25 16:35:00,Melbourne,100,12,120,Round 1,2017,0,90,12,0,2017-04-15 15:59:16,13,Full Time,St Kilda,11.0,11
4,1,15,5,1490432700,2017-03-25 20:05:00,8,Brisbane Lions,+11:00,Carrara,2017-03-25 19:05:00,Brisbane Lions,100,12,98,Round 1,2017,0,96,8,0,2017-04-15 15:59:16,14,Full Time,Gold Coast,2.0,2


In [7]:
def add_game_id(player_stats, game):
    from tqdm import tqdm
    tqdm.pandas()

    player_stats['game_id'] = ''
    for i, row in tqdm(player_stats.iterrows()):
        year = row['year']
        round = row['round']
        if round in ['Qualifying Final', 'Elimination Final']:
            round = 'Finals Week 1'
        round = 'Grand Final' if round == 'Grand Final Replay' else round
        players_team = row['team']
        players_team = 'Brisbane Lions' if 'Brisbane' in players_team else players_team
        players_team = 'Greater Western Sydney' if players_team == 'GWS Giants' else players_team
        opponents_team = row['opponent']
        opponents_team = 'Brisbane Lions' if 'Brisbane' in opponents_team else opponents_team
        opponents_team = 'Greater Western Sydney' if opponents_team == 'GWS Giants' else opponents_team

        matched_game = game[(game['year'] == year) & 
                            (game['round'] == round) & 
                            ((game['hteam'] == players_team) | (game['hteam'] == opponents_team))]

        if not matched_game.empty:
            player_stats.at[i, 'game_id'] = matched_game['game_id'].values[0]
            # print(matched_game['game_id'].values[0])


0it [00:00, ?it/s]

238965it [10:42, 371.76it/s]


In [8]:
# investigate missing game_id
player_stats[player_stats['game_id'] == '']
# Yay all games have been matched

,round,year,player_id,player_name,team,opponent,kicks,marks,hand_balls,disp,goals,behinds,hit_outs,tackles,rebounds,inside_50,clearances,clangers,frees_for,frees_against,brownlow,contested_possessions,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,percent_time_played,game_id


In [39]:
# What datatype is game_id?
player_stats['game_id'].dtype

# Change game_id to int
player_stats['game_id'] = player_stats['game_id'].astype(int)

dtype('int64')

In [40]:
temp = player_stats.copy()

In [49]:
# Now create a new schema for the player_stats table
PLAYER_STATS_TABLE = """
CREATE TABLE player_stats (
    round INT,
    year INT,
    player_id INT,
    player_name VARCHAR(255),
    team VARCHAR(50),
    opponent VARCHAR(50),
    kicks INT,
    marks INT,
    hand_balls INT,
    disp INT, -- should be kicks + hand_balls
    goals INT, -- goals are worth 6 points
    behinds INT, -- behinds are worth 1 point
    hit_outs INT, -- ruckmen mostly only get hit outs
    tackles INT,
    rebounds INT,
    inside_50 INT,
    clearances INT,
    clangers INT,
    frees_for INT,
    frees_against INT,
    brownlow INT, -- most brownlow votes in a season without suspension wins the brownlow medal
    contested_possessions INT,
    uncontested_possessions INT,
    contested_marks INT,
    marks_inside_50 INT,
    one_percenters INT,
    bounces INT,
    goal_assists INT,
    percent_time_played INT,
    game_id INT,
    PRIMARY KEY (round, year, player_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id)
);
"""

# drop the old player_stats table
conn.execute("DROP TABLE IF EXISTS player_stats")

# create the new player_stats table
conn.execute(PLAYER_STATS_TABLE)

In [50]:
# insert the data into the new player_stats table
player_stats.to_sql("player_stats", conn, if_exists='append', index=False)

238965

In [18]:
# Join the player_stats table with the game table on game_id
player_stats = pd.read_sql_query("SELECT * FROM player_stats JOIN game ON player_stats.game_id = game.game_id", conn)
player_stats

,round,year,player_id,player_name,team,opponent,kicks,marks,hand_balls,disp,goals,behinds,hit_outs,tackles,rebounds,inside_50,clearances,clangers,frees_for,frees_against,brownlow,contested_possessions,uncontested_possessions,contested_marks,marks_inside_50,one_percenters,bounces,goal_assists,percent_time_played,game_id,round,agoals,game_id,unixtime,date,hteamid,ateam,tz,venue,localtime,winner,complete,hbehinds,ascore,roundname,year,is_final,hscore,abehinds,is_grand_final,updated,hgoals,timestr,hteam,winnerteamid,ateamid
0,1,1996,720,Alastair Clarkson,Melbourne,Geelong,19,6,5,24,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172,1,27,22172,828090600,1996-03-29 20:10:00,11,Geelong,+11:00,M.C.G.,1996-03-29 20:10:00,Geelong,100,8,183,Round 1,1996,0,56,21,0,2022-11-10 21:19:16,8,None,Melbourne,7.0,7
1,1,1996,718,David Cockatoo-Collins,Melbourne,Geelong,3,0,2,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22172,1,27,22172,828090600,1996-03-29 20:10:00,11,Geelong,+11:00,M.C.G.,1996-03-29 20:10:00,Geelong,100,8,183,Round 1,1996,0,56,21,0,2022-11-10 21:19:16,8,None,Melbourne,7.0,7
2,1,1996,719,Don Cockatoo-Collins,Melbourne,Geelong,5,3,1,6,0,0,0,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172,1,27,22172,828090600,1996-03-29 20:10:00,11,Geelong,+11:00,M.C.G.,1996-03-29 20:10:00,Geelong,100,8,183,Round 1,1996,0,56,21,0,2022-11-10 21:19:16,8,None,Melbourne,7.0,7
3,1,1996,715,Greg Doyle,Melbourne,Geelong,8,4,1,9,4,1,7,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,22172,1,27,22172,828090600,1996-03-29 20:10:00,11,Geelong,+11:00,M.C.G.,1996-03-29 20:10:00,Geelong,100,8,183,Round 1,1996,0,56,21,0,2022-11-10 21:19:16,8,None,Melbourne,7.0,7
4,1,1996,712,Damien Gaspar,Melbourne,Geelong,4,3,3,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22172,1,27,22172,828090600,1996-03-29 20:10:00,11,Geelong,+11:00,M.C.G.,1996-03-29 20:10:00,Geelong,100,8,183,Round 1,1996,0,56,21,0,2022-11-10 21:19:16,8,None,Melbourne,7.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238960,Grand Final,2023,12372,Darcy Moore,Collingwood,Brisbane Lions,4,2,3,7,0,0,0,0,3,0,0,2,0,2,0,2,4,0,0,8,0,0,95,34682,Grand Final,13,34682,1696048200,2023-09-30 14:30:00,4,Brisbane Lions,+10:00,M.C.G.,2023-09-30 14:30:00,Collingwood,100,18,86,Grand Final,2023,6,90,8,1,2023-09-30 17:21:43,12,Full Time,Collingwood,4.0,2
238961,Grand Final,2023,12673,Nathan Murphy,Collingwood,Brisbane Lions,4,1,0,4,0,0,0,0,1,0,0,1,1,0,0,1,4,0,0,1,0,0,20,34682,Grand Final,13,34682,1696048200,2023-09-30 14:30:00,4,Brisbane Lions,+10:00,M.C.G.,2023-09-30 14:30:00,Collingwood,100,18,86,Grand Final,2023,6,90,8,1,2023-09-30 17:21:43,12,Full Time,Collingwood,4.0,2
238962,Grand Final,2023,4182,Scott Pendlebury,Collingwood,Brisbane Lions,16,5,8,24,1,0,0,4,1,4,6,3,1,0,0,9,13,0,1,2,0,0,77,34682,Grand Final,13,34682,1696048200,2023-09-30 14:30:00,4,Brisbane Lions,+10:00,M.C.G.,2023-09-30 14:30:00,Collingwood,100,18,86,Grand Final,2023,6,90,8,1,2023-09-30 17:21:43,12,Full Time,Collingwood,4.0,2
238963,Grand Final,2023,12740,Isaac Quaynor,Collingwood,Brisbane Lions,10,5,2,12,0,0,0,3,1,0,0,0,0,0,0,4,9,0,0,2,0,1,86,34682,Grand Final,13,34682,1696048200,2023-09-30 14:30:00,4,Brisbane Lions,+10:00,M.C.G.,2023-09-30 14:30:00,Collingwood,100,18,86,Grand Final,2023,6,90,8,1,2023-09-30 17:21:43,12,Full Time,Collingwood,4.0,2


In [53]:
# Export the schema of the afl.db
os.system("sqlite3 ../afl.db .schema > ../afl_schema.sql")

0

In [52]:
# Import the schema from the edited afl_schema.sql
os.system("sqlite3 ../afl.db < ../afl_schema.sql")

1